In [55]:
import pandas as pd
import sqlite3

In [56]:
# run transform_and_create_table.ipynb

In [57]:
conn = sqlite3.connect('teste_ailos.db')

base_originacao = pd.read_sql_query("SELECT * FROM base_originacao", conn)
base_carteira = pd.read_sql_query("SELECT * FROM base_carteira", conn)

In [58]:
%reload_ext sql
%sql sqlite:///teste_ailos.db

# RESOLUÇÃO DO EXERCÍCIO EM SQL
Em uma visão mensal (DTBASE), gere uma  tabela consolidada de carteira, com ordenação crescente pela DTBASE, com as seguintes informações:
- datas bases
- quantidade de cooperativas distintas
- quantidade de CPFs distintos
- soma dos saldo devedor
- média dos dias de atraso
- média da quantidade de parcelas
- menos data de originação dos contratos
- contrato com maior número de parcelas
- prazo médio ponderado dos contratos (valor da originação e quantidade de parcelas)
- taxa de contratos com atrasos > 90
- taxa de saldo devedor com atraso > 90 dias

(ou seja, cada linha deverá ter os 11 itens mencionados, e serão 12 linhas na tabela)

In [59]:
%%sql
select CONTRATO, QTPARCELAS from base_originacao where QTPARCELAS = (select max(QTPARCELAS) from base_originacao)

 * sqlite:///teste_ailos.db
Done.


CONTRATO,QTPARCELAS
498576,240.0
301340,240.0
214394,240.0
425479,240.0
468921,240.0
561730,240.0
486893,240.0
542421,240.0
549274,240.0
474081,240.0


In [154]:
%%sql
with all_data as (
    select 
        bc.*
        ,bo.QTPARCELAS
        ,iif(QTDIASATRASO > 90, 1, 0) is_novent_dias_atraso
    from 
        base_carteira bc
    inner join 
        base_originacao bo 
            on bc.CONTRATO = bo.CONTRATO
            and bc.CPF = bo.CPF
            and bc.CONTA = bo.CONTA
            and bc.COOPERATIVA = bo.COOPERATIVA
            and bc.DTCONTRATO = bo.DTCONTRATO
            and bc.VLCONTRATO = bo.VLCONTRATO
    order by 
        date(DTBASE) asc
)

select 
    date(DTBASE) as dt_base
    ,count(distinct COOPERATIVA) as qnt_coop
    ,count(distinct CPF) as qnt_cpfs
    ,round(sum(SDDEVEDOR), 2) as sum_sddev
    ,round(avg(QTDIASATRASO), 2) as avg_dias_atraso
    ,round(avg(QTPARCELAS), 2) as avg_qnt_parcelas
    ,min(date(DTCONTRATO)) as min_dt_contrato
    ,(select min(date(DTCONTRATO)) from all_data) as min_dt_contrato_ever
--  ,contrato com maior número de parcelas  
    ,round(sum(VLCONTRATO * QTPARCELAS) / sum(VLCONTRATO), 2) as avg_pond_prazo
    ,round(((count(iif(QTDIASATRASO > 90, 1, Null)) / cast(count(distinct CONTRATO) as float))) * 100, 2) as percent_contrat_novent_dias
    ,round((sum(iif(QTDIASATRASO > 90, SDDEVEDOR, 0)) / sum(SDDEVEDOR)) * 100, 2) as percent_sddev_novent_dias
from
    all_data
group by 
    date(DTBASE)
order by 
    date(DTBASE) asc


 * sqlite:///teste_ailos.db
Done.


dt_base,qnt_coop,qnt_cpfs,sum_sddev,avg_dias_atraso,avg_qnt_parcelas,min_dt_contrato,min_dt_contrato_ever,avg_pond_prazo,percent_contrat_novent_dias,percent_sddev_novent_dias
2021-01-31,5,3333,13215691.2,22.86,37.0,2013-01-24,2013-01-24,61.9,9.51,3.88
2021-02-28,4,3334,13058845.8,21.79,37.16,2013-01-24,2013-01-24,60.95,9.03,3.69
2021-03-31,5,3599,14389670.4,21.99,37.1,2013-01-24,2013-01-24,63.01,8.86,3.84
2021-04-30,5,3704,15181006.8,21.97,37.23,2013-01-24,2013-01-24,64.66,9.13,3.8
2021-05-31,4,571,2399193.0,28.37,37.71,2014-07-15,2013-01-24,61.57,12.61,5.8
2021-06-30,5,3918,15556938.0,24.62,37.0,2013-01-24,2013-01-24,62.01,10.36,4.38
2021-07-31,5,4011,16769962.8,24.97,37.41,2013-01-24,2013-01-24,69.34,10.6,4.47
2021-08-31,5,4119,17236592.4,26.26,38.34,2013-01-24,2013-01-24,69.31,11.48,4.96
2021-09-30,5,4313,17437828.2,27.24,37.88,2013-01-24,2013-01-24,69.05,11.92,5.14
2021-10-31,5,4395,17903427.6,30.41,38.02,2013-01-24,2013-01-24,69.32,12.72,5.24


In [107]:
%%sql
select 
    bc.*
    ,bo.QTPARCELAS
from 
    base_carteira bc
inner join 
    base_originacao bo 
        on bc.CONTRATO = bo.CONTRATO
        and bc.CPF = bo.CPF
        and bc.CONTA = bo.CONTA
        and bc.COOPERATIVA = bo.COOPERATIVA
        and bc.DTCONTRATO = bo.DTCONTRATO
        and bc.VLCONTRATO = bo.VLCONTRATO
order by 
    date(DTBASE) asc
limit 10

 * sqlite:///teste_ailos.db
Done.


index,DTBASE,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,SDDEVEDOR,QTDIASATRASO,SDATRASO,VLPROVISAO,QTPARCELAS
0,2021-01-31 00:00:00,A,154,49834,349384,Empréstimo,2020-08-25 00:00:00,9761.4,9557.4,0.0,0.0,30.6,72.0
1,2021-01-31 00:00:00,A,186,398914,444074,Empréstimo,2021-01-08 00:00:00,4314.6,4365.6,0.0,0.0,81.6,36.0
2,2021-01-31 00:00:00,A,430,549582,372569,Empréstimo,2020-10-02 00:00:00,5548.8,5273.4,0.0,0.0,20.4,48.0
3,2021-01-31 00:00:00,A,512,217818,223696,Empréstimo,2020-01-31 00:00:00,3111.0,2539.8,0.0,0.0,10.2,48.0
4,2021-01-31 00:00:00,A,754,546819,435007,Empréstimo,2020-12-23 00:00:00,601.8,601.8,19.0,601.8,10.2,12.0
5,2021-01-31 00:00:00,A,1161,199810,320879,Empréstimo,2020-07-03 00:00:00,2774.4,2448.0,0.0,0.0,10.2,36.0
6,2021-01-31 00:00:00,A,1180,330757,355644,Empréstimo,2020-08-28 00:00:00,5120.4,4212.6,0.0,0.0,10.2,24.0
7,2021-01-31 00:00:00,A,1240,522707,278328,Empréstimo,2020-04-20 00:00:00,14932.8,12668.4,0.0,0.0,40.8,48.0
8,2021-01-31 00:00:00,A,1491,518714,203568,Empréstimo,2019-11-27 00:00:00,1101.6,581.4,0.0,0.0,30.6,24.0
9,2021-01-31 00:00:00,A,2191,613085,241210,Empréstimo,2020-03-16 00:00:00,1326.0,1326.0,136.0,1326.0,520.2,48.0


In [61]:
base_originacao.sort_values(by='QTPARCELAS', ascending=False)

,index,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,VLPARCELA,QTPARCELAS
966,966,A,115218,654272,498585,Financiamento,2021-05-04 00:00:00,93891.0,1224.0,240.0
2495,2495,A,299048,193571,228021,Financiamento,2020-06-08 00:00:00,107497.8,1264.8,240.0
5787,5787,A,695718,223059,426321,Financiamento,2021-01-22 00:00:00,110200.8,1672.8,240.0
5318,5318,A,638617,299490,370404,Financiamento,2021-01-22 00:00:00,80172.0,1060.8,240.0
2496,2496,A,299100,298042,377690,Financiamento,2020-11-27 00:00:00,127693.8,1662.6,240.0
...,...,...,...,...,...,...,...,...,...,...
1843,1843,A,224948,236477,579480,Financiamento,2021-12-16 00:00:00,4671.6,0.0,0.0
6137,6137,B,179077,116704,101794,Financiamento,2021-10-20 00:00:00,846.6,0.0,0.0
1443,1443,A,174684,228084,669308,Financiamento,2021-10-25 00:00:00,4243.2,0.0,0.0
939,939,A,112811,699085,734638,Financiamento,2021-12-29 00:00:00,12729.6,0.0,0.0


In [62]:
base_carteira[base_carteira.CONTRATO == 498576].sort_values(by='DTBASE')

,index,DTBASE,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,SDDEVEDOR,QTDIASATRASO,SDATRASO,VLPROVISAO
10271,10271,2021-04-30 00:00:00,A,1142,126440,498576,Financiamento,2021-04-05 00:00:00,106681.8,107426.4,0.0,0.0,316.2
18463,18463,2021-07-31 00:00:00,A,1142,126440,498576,Financiamento,2021-04-05 00:00:00,106681.8,107089.8,0.0,0.0,316.2
22474,22474,2021-08-31 00:00:00,A,1142,126440,498576,Financiamento,2021-04-05 00:00:00,106681.8,107018.4,0.0,0.0,316.2
26594,26594,2021-09-30 00:00:00,A,1142,126440,498576,Financiamento,2021-04-05 00:00:00,106681.8,106906.2,0.0,0.0,316.2
30908,30908,2021-10-31 00:00:00,A,1142,126440,498576,Financiamento,2021-04-05 00:00:00,106681.8,106804.2,0.0,0.0,316.2
35304,35304,2021-11-30 00:00:00,A,1142,126440,498576,Financiamento,2021-04-05 00:00:00,106681.8,106722.6,0.0,0.0,316.2
